In [1]:
%%time
%load_ext autoreload
%autoreload 2

# import sys
import os
import warnings

# from urllib.request import urlretrieve
# from argparse import Namespace
# from importlib import reload


# import numpy as np
# import pandas as pd
import scanpy as sc
# import matplotlib.pyplot as plt
# import seaborn as sns
# import networkx as nx

# from sklearn.decomposition import _nmf as sknmf
from sklearn.exceptions import ConvergenceWarning
# from scipy.stats import rankdata
# from scipy.cluster import hierarchy

from gepdynamics import _utils
# from gepdynamics import _constants
# from gepdynamics import cnmf
# from gepdynamics import pfnmf
from gepdynamics import comparator

# Move to the project's home directory, as defined in _constants
_utils.cd_proj_home()
print(os.getcwd())

/cs/labs/mornitzan/yotamcon/gep-dynamics
CPU times: user 2.87 s, sys: 1.02 s, total: 3.89 s
Wall time: 5.6 s


In [4]:
%%time
results_dir = _utils.set_dir('results')

split_adatas_dir = _utils.set_dir(results_dir.joinpath('marjanovic_mmLungPlate_split'))

k_12 = sc.read_h5ad(split_adatas_dir.joinpath('04_K_12w_ND_GEPs.h5ad'))

k_30 = sc.read_h5ad(split_adatas_dir.joinpath('05_K_30w_ND_GEPs.h5ad'))

kp_12 = sc.read_h5ad(split_adatas_dir.joinpath('06_KP_12w_ND_GEPs.h5ad'))

kp_30 = sc.read_h5ad(split_adatas_dir.joinpath('08_KP_30w_ND_GEPs.h5ad'))

CPU times: user 646 ms, sys: 286 ms, total: 933 ms
Wall time: 8.36 s


### Comparing Kras 12 and 30 weeks

In [5]:

pairs = [(k_12, k_30), (k_30, k_12), (kp_12, kp_30), (kp_30, kp_12), (k_12, kp_12), (k_30, kp_30)]

for adata_a, adata_b in pairs:
    comparison_dir = results_dir.joinpath(f"{adata_a.uns['sname']}_{adata_b.uns['sname']}")

    tst = comparator.Comparator(
        adata_a, adata_a.obsm['usages'], adata_b, comparison_dir, 'torchnmf',
        max_nmf_iter=800, verbosity=1, )

    with warnings.catch_warnings():  # supress convergence warning
        warnings.simplefilter(action='ignore', category=ConvergenceWarning)
        tst.extract_geps_on_jointly_hvgs()

    tst.decompose_b(repeats=10)

    tst.plot_loss_per_cell_histograms(show=False)
    tst.plot_usages_clustermaps(show=False)
    tst.plot_decomposition_comparisons(show=False)
    tst.run_gsea(gprofiler_kwargs=dict(organism='hsapiens', sources=['GO:BP', 'WP', 'REAC', 'KEGG']))

    tst.calculate_fingerprints()

    tst.print_errors()

    tst.save_to_file(comparison_dir.joinpath('comparator.npz'))

Directory "results/t04_t05" does not exist.
trying to create it at /cs/labs/mornitzan/yotamcon/gep-dynamics/results/t04_t05


/cs/labs/mornitzan/yotamcon/miniconda3/envs/pp_t/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/cs/labs/mornitzan/yotamcon/miniconda3/envs/pp_t/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


Extracting A GEPs on jointly highly variable genes
Decomposing B using A GEPs and no additional GEPs
beta 1 loss = 410354.625, # iterations was 50
Decomposing B de-novo
beta 1 loss = 356336.0, # iterations was 420
beta 1 loss = 349574.84375, # iterations was 520
beta 1 loss = 342159.375, # iterations was 590
beta 1 loss = 336335.71875, # iterations was 650
Decomposing B using A GEPs and up to 3 additional GEPs
Working on added rank = 1
repeat 0, after 96 iterations reached error =  393940.8
repeat 3, after 113 iterations reached error =  393925.0
Working on added rank = 2
repeat 0, after 184 iterations reached error =  385748.2
repeat 1, after 228 iterations reached error =  385596.4
repeat 2, after 176 iterations reached error =  384935.9
repeat 3, after 180 iterations reached error =  384486.1
repeat 4, after 205 iterations reached error =  383882.4
repeat 8, after 287 iterations reached error =  383740.5
Working on added rank = 3
repeat 0, after 525 iterations reached error =  37471

/cs/labs/mornitzan/yotamcon/miniconda3/envs/pp_t/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/cs/labs/mornitzan/yotamcon/miniconda3/envs/pp_t/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


Extracting A GEPs on jointly highly variable genes
Decomposing B using A GEPs and no additional GEPs
beta 1 loss = 370878.03125, # iterations was 70
Decomposing B de-novo
beta 1 loss = 309835.125, # iterations was 530
beta 1 loss = 303844.0, # iterations was 430
beta 1 loss = 299713.875, # iterations was 660
beta 1 loss = 294143.8125, # iterations was 530
Decomposing B using A GEPs and up to 3 additional GEPs
Working on added rank = 1
repeat 0, after 160 iterations reached error =  360975.2
repeat 1, after 116 iterations reached error =  360644.9
repeat 2, after 139 iterations reached error =  360608.1
repeat 3, after 149 iterations reached error =  360533.2
repeat 4, after 190 iterations reached error =  360523.1
repeat 6, after 146 iterations reached error =  360336.6
Working on added rank = 2
repeat 0, after 177 iterations reached error =  352204.5
Working on added rank = 3
repeat 0, after 260 iterations reached error =  345963.2
repeat 2, after 214 iterations reached error =  34573

...

Extracting A GEPs on jointly highly variable genes
Decomposing B using A GEPs and no additional GEPs
beta 1 loss = 371216.0, # iterations was 60
Decomposing B de-novo
beta 1 loss = 296165.125, # iterations was 520
beta 1 loss = 290011.0, # iterations was 400
beta 1 loss = 284879.28125, # iterations was 350
beta 1 loss = 279788.5, # iterations was 470
Decomposing B using A GEPs and up to 3 additional GEPs
Working on added rank = 1
repeat 0, after 92 iterations reached error =  359892.6
repeat 1, after 136 iterations reached error =  359646.2
repeat 2, after 291 iterations reached error =  354992.7
repeat 5, after 225 iterations reached error =  354802.3
Working on added rank = 2
repeat 0, after 268 iterations reached error =  344153.1
repeat 1, after 305 iterations reached error =  343153.4
repeat 6, after 246 iterations reached error =  343032.5
Working on added rank = 3
repeat 0, after 177 iterations reached error =  335717.2
repeat 1, after 404 iterations reached error =  334636.4
re

/cs/labs/mornitzan/yotamcon/miniconda3/envs/pp_t/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/cs/labs/mornitzan/yotamcon/miniconda3/envs/pp_t/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


Extracting A GEPs on jointly highly variable genes
Decomposing B using A GEPs and no additional GEPs
beta 1 loss = 401176.625, # iterations was 50
Decomposing B de-novo
beta 1 loss = 327027.9375, # iterations was 460
beta 1 loss = 319423.875, # iterations was 720
beta 1 loss = 313297.71875, # iterations was 620
beta 1 loss = 308781.71875, # iterations was 610
Decomposing B using A GEPs and up to 3 additional GEPs
Working on added rank = 1
repeat 0, after 103 iterations reached error =  380741.3
repeat 1, after 87 iterations reached error =  380513.2
Working on added rank = 2
repeat 0, after 157 iterations reached error =  360460.7
repeat 1, after 107 iterations reached error =  360235.2
repeat 2, after 148 iterations reached error =  360163.7
repeat 3, after 159 iterations reached error =  360153.6
repeat 5, after 110 iterations reached error =  360119.1
Working on added rank = 3
repeat 0, after 257 iterations reached error =  349083.9
repeat 7, after 348 iterations reached error =  34

/cs/labs/mornitzan/yotamcon/miniconda3/envs/pp_t/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/cs/labs/mornitzan/yotamcon/miniconda3/envs/pp_t/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


Extracting A GEPs on jointly highly variable genes
Decomposing B using A GEPs and no additional GEPs
beta 1 loss = 1049843.0, # iterations was 60
Decomposing B de-novo
beta 1 loss = 895132.375, # iterations was 800
beta 1 loss = 884445.25, # iterations was 800
beta 1 loss = 873406.25, # iterations was 690
beta 1 loss = 859591.4375, # iterations was 800
Decomposing B using A GEPs and up to 3 additional GEPs
Working on added rank = 1
repeat 0, after 80 iterations reached error =  1011733.9
repeat 1, after 84 iterations reached error =  1011683.7
repeat 3, after 98 iterations reached error =  1011678.0
repeat 6, after 85 iterations reached error =  1011674.6
repeat 8, after 81 iterations reached error =  1011635.3
Working on added rank = 2
repeat 0, after 134 iterations reached error =  982723.6
repeat 1, after 146 iterations reached error =  982714.9
repeat 6, after 130 iterations reached error =  982620.5
Working on added rank = 3
repeat 0, after 295 iterations reached error =  958239.5